In [1]:
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
mean = [0.4914, 0.4822, 0.4465]
std = [0.2023, 0.1994, 0.2010]

In [3]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=0, translate=(0.1,0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])

In [4]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])

In [5]:
train_dataset = datasets.CIFAR10(root='D:/STUDY/MS/DATS 6450 Computer Vision', train=True, download=False, transform=transform_train)

In [6]:
test_dataset = datasets.CIFAR10(root='D:/STUDY/MS/DATS 6450 Computer Vision', train=False, download=False, transform=transform_test)

In [7]:
valid_dataset, train_dataset = torch.utils.data.random_split(train_dataset, [5000, len(train_dataset) - 5000])

In [8]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
val_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)

In [24]:
base_hidden_units = 32
weight_decay = 1e-4

class CIFAR10CNN(nn.Module):
    def __init__(self):
        super(CIFAR10CNN, self).__init__()
        #CONV1
        self.conv1 = nn.Conv2d(3,base_hidden_units,3, padding=1)
        self.bn1 = nn.BatchNorm2d(base_hidden_units)

        #CONV2
        self.conv2 = nn.Conv2d(base_hidden_units,base_hidden_units,3, padding=1)
        self.bn2 = nn.BatchNorm2d(base_hidden_units)

        #CONV3
        self.conv3 = nn.Conv2d(base_hidden_units,base_hidden_units*2,3, padding=1)
        self.bn3 = nn.BatchNorm2d(base_hidden_units*2)

        #CONV4
        self.conv4 = nn.Conv2d(base_hidden_units*2,base_hidden_units*2,3, padding=1)
        self.bn4 = nn.BatchNorm2d(base_hidden_units*2)

        #CONV5
        self.conv5 = nn.Conv2d(base_hidden_units*2,base_hidden_units*4,3, padding=1)
        self.bn5 = nn.BatchNorm2d(base_hidden_units*4)

        #CONV6
        self.conv6 = nn.Conv2d(base_hidden_units*4,base_hidden_units*4,3, padding=1)
        self.bn6 = nn.BatchNorm2d(base_hidden_units*4)

        #FC1
        self.fc1 = nn.Linear(base_hidden_units*4*4*4, 10)

        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.4)
        
        self.pool = nn.MaxPool2d(2,2)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))

        x = self.dropout1(self.pool(x))

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))

        x = self.dropout2(self.pool(x))

        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))

        x = self.dropout3(self.pool(x))

        x = x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x



In [25]:
model = CIFAR10CNN()
model

CIFAR10CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_

In [26]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
device

'cuda'

In [27]:
model.to(device)

CIFAR10CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_

In [28]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

In [29]:
# Training loop
def train(model, train_loader, valid_loader, criterion, optimizer, epochs=125):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{epochs}, Training Loss: {epoch_loss:.4f}')

        # Validate the model
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Validation Accuracy: {100 * correct / total:.2f}%')

In [15]:
train_loader

In [30]:
train(model, train_loader, val_loader, criterion, optimizer)

Epoch 1/125, Training Loss: 1.5916
Validation Accuracy: 51.90%
Epoch 2/125, Training Loss: 1.2227
Validation Accuracy: 55.80%
Epoch 3/125, Training Loss: 1.0546
Validation Accuracy: 66.84%
Epoch 4/125, Training Loss: 0.9573
Validation Accuracy: 70.54%
Epoch 5/125, Training Loss: 0.8824
Validation Accuracy: 71.60%
Epoch 6/125, Training Loss: 0.8347
Validation Accuracy: 72.38%
Epoch 7/125, Training Loss: 0.7935
Validation Accuracy: 74.50%
Epoch 8/125, Training Loss: 0.7570
Validation Accuracy: 76.52%
Epoch 9/125, Training Loss: 0.7334
Validation Accuracy: 77.00%
Epoch 10/125, Training Loss: 0.7048
Validation Accuracy: 77.78%
Epoch 11/125, Training Loss: 0.6832
Validation Accuracy: 77.70%
Epoch 12/125, Training Loss: 0.6656
Validation Accuracy: 76.32%
Epoch 13/125, Training Loss: 0.6470
Validation Accuracy: 78.28%
Epoch 14/125, Training Loss: 0.6350
Validation Accuracy: 78.58%
Epoch 15/125, Training Loss: 0.6215
Validation Accuracy: 79.22%
Epoch 16/125, Training Loss: 0.6106
Validation Ac

In [31]:
import matplotlib.pyplot as plt

plt.plot(epoch_loss, label='training loss')
plt.show()

NameError: name 'epoch_loss' is not defined